In [ ]:
import requests
import random
from datetime import datetime, timedelta, time
from collections import defaultdict
import math
# import psycopg2
import pandas as pd
from sqlalchemy import create_engine,text

In [ ]:
class Matakuliah:
    def __init__(self, nama, sks, dosen, semester, prodi, tipe_ruang_required=None):
        self.nama = nama
        self.sks = sks
        self.dosen = dosen
        self.semester = semester
        self.prodi = prodi
        self.tipe_ruang_required = tipe_ruang_required if tipe_ruang_required else []

class Ruang:
    def __init__(self, nama, tipe_ruang):
        self.nama = nama
        self.tipe_ruang = tipe_ruang if isinstance(tipe_ruang, list) else [tipe_ruang]
        


In [ ]:
class PenjadwalanDosen:
    def __init__(self):
          self.dosen_list = []
          self.matakuliah_list = []
          self.ruang_list = []
          self.hari_list = ['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat']
          self.jam_mulai = datetime.strptime("08:00", "%H:%M")
          self.jam_selesai = datetime.strptime("16:00", "%H:%M")
          self.durasi_slot = timedelta(minutes=45)
          self.istirahat = [
              (datetime.strptime("10:00", "%H:%M"), datetime.strptime("10:30", "%H:%M")),
              (datetime.strptime("12:00", "%H:%M"), datetime.strptime("13:00", "%H:%M"))
          ]
          self.jam_list = self.generate_jam_list()
          print(len(self.jam_list))
          self.prodi_jadwal = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))

    def baca_datamk(self):
        # baca dari database tabel perkuliahan dengan id generate 1 --belum
        db = 'mysql+pymysql://root:@localhost/db_optimasi1'
        engine = create_engine(db)
        query = text("""
            SELECT tb_rombel.nama_kelas AS kelas,
                tb_matakuliah.nama_matakuliah AS matakuliah,
                tb_dosen.nama_dosen AS dosen,
                tb_matakuliah.sks AS sks,
                tb_matakuliah.status AS status,
                tb_matakuliah.kategori AS kategori,
                id_perkuliahan,
                id_semester
            FROM tb_perkuliahan
            JOIN tb_matakuliah ON tb_perkuliahan.kode_matakuliah = tb_matakuliah.kode_matakuliah
            JOIN tb_rombel ON tb_perkuliahan.id_kelasrombel = tb_rombel.id_kelasrombel
            JOIN tb_dosen ON tb_perkuliahan.kode_dosen = tb_dosen.kode_dosen
            JOIN tb_generate 
            ON tb_generate.id_generate = tb_rombel.id_generate 
            AND tb_generate.id_generate = tb_matakuliah.id_generate 
            AND tb_generate.id_generate = tb_dosen.id_generate
            AND tb_generate.id_generate = 'G0706-02'
            LIMIT 40
        """)
        with engine.connect() as connection:
            df_matkul = pd.read_sql_query(query, connection)

        # Iterasi per baris untuk menambahkan data
        for _, row in df_matkul.iterrows():
            self.tambah_matkul(row['matakuliah'], row['dosen'], row['sks'], row['kelas'], row['status'], row['id_perkuliahan'],row['id_semester'],row['kategori'])
            

    def baca_dataruang(self):
        # berasal dari database dengan id generate 1
        # Iterasi per baris untuk menambahkan data
        db = 'mysql+pymysql://root:@localhost/db_optimasi1'
        engine = create_engine(db)
        query = text("""
        SELECT nama_ruangan, 
            status_ruangan 
        FROM tb_ruang 
        JOIN tb_generate ON tb_ruang.id_generate = tb_generate.id_generate
        AND tb_generate.id_generate = 'G0706-02'
        LIMIT 40
        """)
        with engine.connect() as connection:
            df_ruang = pd.read_sql_query(query, connection)

        # Iterasi per baris untuk menambahkan data
        for _, row in df_ruang.iterrows():
            self.tambah_ruang(row['nama_ruangan'], row['status_ruangan'])
    
    def tambah_matakuliah(self, nama, sks, dosen, semester, prodi, tipe_ruang_required=None):
        matakuliah = Matakuliah(nama, sks, dosen, semester, prodi, tipe_ruang_required)
        self.matakuliah_list.append(matakuliah)
        return matakuliah

    def tambah_ruang(self, nama, kapasitas, tipe_ruang):
        ruang = Ruang(nama, kapasitas, tipe_ruang)
        self.ruang_list.append(ruang)
        return ruang